In [1]:
!mkdir -p input
!mkdir -p workflows
!echo "A" > input/sample_A.fastq
!echo "B" > input/sample_B.fastq
!echo "C" > input/sample_C.fastq

In [4]:
%%file test.nf
num = Channel.from( 1, 2, 3 )

process basicExample {
  input:
  val x from num

  "echo process job $x"
}

Overwriting test.nf


In [7]:
%%bash
export JAVA_CMD=/home/studio-lab-user/.conda/envs/pipeline_tutorial/bin/java
/home/studio-lab-user/.conda/envs/pipeline_tutorial/bin/nextflow test.nf -dsl1

N E X T F L O W  ~  version 22.04.0
Launching `test.nf` [distraught_dijkstra] DSL1 - revision: bb35bbac74
[-        ] process > basicExample -

executor >  local (3)
[94/37e31a] process > basicExample (2) [100%] 3 of 3 ✔

executor >  local (3)
[94/37e31a] process > basicExample (2) [100%] 3 of 3 ✔



In [8]:
%%file test.nf
 
process bwa_map {
    input:
        tuple val(sampleName), path(reads)
    output:
        tuple  val(sampleName), path("sample_${sampleName}.bam")
    script:
        """
        cat  $reads > sample_${sampleName}.bam
        sleep 5
        """
}

process samtools_sort {
    input:
        tuple val(sampleName), path(unsorted_bam)
    output:
        tuple  val(sampleName), path("sample_sorted_${sampleName}.bam")
    script:
        """
        cat $unsorted_bam > sample_sorted_${sampleName}.bam
        sleep 5
        """
}

process samtools_index {
    input:
        tuple val(sampleName), path(sorted_bam)
    output:
        tuple  val(sampleName), path("sample_sorted_${sampleName}.bam.bai")
    script:
        """
        cat $sorted_bam > sample_sorted_${sampleName}.bam.bai
        sleep 5
        """
}

process genotype_calling {
    input:
        file "*.bam"
        file "*.bai"
    output:
        publishDir "${baseDir}/final_output", mode: 'copy'
        path("all.vcf")
    script:
        """
        cat *.bam > all.vcf
        """  
}

workflow {
    paths = Channel.from(
               tuple("A", "${baseDir}/input/sample_A.fastq"),
               tuple("B", "${baseDir}/input/sample_B.fastq"),
               tuple("C", "${baseDir}/input/sample_C.fastq"))
        .map(row -> tuple(row[0], row[1]))
    unsorted_bams_ch = bwa_map(paths)
    sorted_bams_ch = samtools_sort(unsorted_bams_ch)
    sorted_index_ch = samtools_index(sorted_bams_ch)
    genotype_calling(
        sorted_bams_ch.map(row -> row[1]).collect(),
        sorted_index_ch.map(row -> row[1]).collect())
}

Overwriting test.nf


## Follow these steps
* Open terminal
* Activate conda environment, e.g., `conda activate notebook-env`
* Change to currect directory
* Run nextflow pipeline using the following command
  `nextflow run -dsl2 test.nf`
* Check output file 
  `cat final_output/all.vcf`~

In [12]:
%%bash
export JAVA_CMD=/home/studio-lab-user/.conda/envs/pipeline_tutorial/bin/java
/home/studio-lab-user/.conda/envs/pipeline_tutorial/bin/nextflow test.nf -dsl2

N E X T F L O W  ~  version 22.04.0
Launching `test.nf` [thirsty_pauling] DSL2 - revision: 2221fbec4e
[-        ] process > bwa_map        -
[-        ] process > samtools_sort  -
[-        ] process > samtools_index -

[-        ] process > bwa_map          -
[-        ] process > samtools_sort    -
[-        ] process > samtools_index   -
[-        ] process > genotype_calling -

executor >  local (3)
[d8/c70777] process > bwa_map (2)      [  0%] 0 of 3
[-        ] process > samtools_sort    -
[-        ] process > samtools_index   -
[-        ] process > genotype_calling -

executor >  local (6)
[d8/c70777] process > bwa_map (2)       [100%] 3 of 3 ✔
[cb/463385] process > samtools_sort (3) [  0%] 0 of 3
[-        ] process > samtools_index    -
[-        ] process > genotype_calling  -

executor >  local (7)
[d8/c70777] process > bwa_map (2)        [100%] 3 of 3 ✔
[60/f29268] process > samtools_sort (2)  [ 33%] 1 of 3
[14/a45c74] process > samtools_index (1) [  0%] 0 of 1
[-        